In [11]:
from agents import Agent, Runner, function_tool, ItemHelpers

@function_tool
def get_weather(city:str):
    print(city)
    return "30 degrees"
agent = Agent(
    name="Assistant Agent",
    instructions="너는도움을 주는 어시스턴트야, 답변할때 필요하면 tool들을 사용해. ",
    tools=[get_weather]
)

# agent가 실시간으로 뭐하는지 업데이트 해준다고..?
stream = Runner.run_streamed(agent,"안녕 잘지냄? 안녕 서울날씨는 어떄?")

async for event in stream.stream_events():
    if event.type == "raw_response_event":
        continue
    elif event.type == "agent_updated_stream_event":
        print("Agent updated to", event.new_agent.name)
    elif event.type == "run_item_stream_event":
        if event.item.type == "tool_call_item":
            print(event.item.raw_item.to_dict())
        elif event.item.type == "tool_call_output_item":
            print(event.item.output)
        elif event.item.type == "message_output_item":
            print(ItemHelpers.text_message_output(event.item))
    print("=" * 20)


Agent updated to Assistant Agent
{'arguments': '{"city":"서울"}', 'call_id': 'call_JfIQkmQBuaudQoZfU1LPaBU0', 'name': 'get_weather', 'type': 'function_call', 'id': 'fc_06f4823b7e7a2dd900698817a6f688819a835555b3bfddd771', 'status': 'completed'}
서울
30 degrees
안녕하세요! 저는 항상 잘 지내고 있어요. 서울의 현재 날씨는 30도예요. 더운 편이니 시원하게 지내세요! 궁금한 점 또 있으시면 말씀해 주세요.


In [ ]:
from agents import Agent, Runner, function_tool, ItemHelpers

@function_tool
def get_weather(city:str):
    print(city)
    return "30 degrees"
agent = Agent(
    name="Assistant Agent",
    instructions="너는도움을 주는 어시스턴트야, 답변할때 필요하면 tool들을 사용해. ",
    tools=[get_weather]
)

message = ""
args = ""

# agent가 실시간으로 뭐하는지 업데이트 해준다고..?
stream = Runner.run_streamed(agent,"안녕 잘지냄? 안녕 서울날씨는 어떄?")

async for event in stream.stream_events():
    if event.type == "raw_response_event":
        event_type = event.data.type
        if event_type == "response.output_text.delta":
            message += event.data.delta
            print(message)
        elif event_type == "response.function_call_arguments.delta":
            args= event.data.delta
            print(args)
        elif event_type == "response.completed":
            message = ""
            args = ""
        #  delta 가 보이면 에이전트가 텍스트를 좀 쓰는거라함
        #  .done 이건 에이전트가 인수들을 쓰는걸 끝낸거


{"
city
":"
Se
oul
"}
Seoul
안
녕하세요
!
 저는
 항상
 잘
 지
내
고
 있어
요
.
 오늘
 서울
의
 날
씨
는
 
30
도
 정도
로
 무
더
운
 편
이에
요
.
 더
운
 날
씨
에
 건강
 조
심
하세요
!
 추가
로
 궁
금
한
 점
 있
으면
 언제
든
 말씀
해
 주세요
.
